## Modelagem

Nesta série de scripts iremos começar o processo de modelagem. O método que mais se aplica ao nosso conjunto de dados é a análise de dados em painel. Aqui iremos aplicar este primeiro método e depois iremos explorar outras opções. Usei o exemplo desse [link](https://bashtage.github.io/linearmodels/panel/examples/examples.html).

In [21]:
# Carregar librarias
import pandas as pd
import statsmodels.api as sm
from linearmodels.datasets import wage_panel
from linearmodels.panel import PooledOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import BetweenOLS
from linearmodels.panel import compare
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Carregar dataset
data_sugarcane = pd.read_csv("dat_join.csv", sep =  ';')

In [3]:
# Visão dos dados
data_sugarcane.head()

,CD_MUN,Município,value,YEAR,AET,NDVI,NPP,SOIL,TMIN,TMMX,PR
0,3500105,Adamantina (SP),50000.0,2000,704,5174,4047,166,163,288,91
1,3500204,Adolfo (SP),NaN,2000,716,4080,5245,255,162,290,94
2,3500303,Aguaí (SP),80000.0,2000,675,5614,8672,399,151,272,109
3,3500550,Águas de Santa Bárbara (SP),NaN,2000,776,5388,8146,246,142,270,95
4,3500709,Agudos (SP),70000.0,2000,746,5969,9080,248,146,274,94


In [4]:
# Converter o ano para categórico
year_sugar = pd.Categorical(data_sugarcane.YEAR)
year_sugar

[2000, 2000, 2000, 2000, 2000, ..., 2020, 2020, 2020, 2020, 2020]
Length: 10185
Categories (21, int64): [2000, 2001, 2002, 2003, ..., 2017, 2018, 2019, 2020]

In [5]:
# Criar o index utilizando o ano e o código do municipio
data_sugar = data_sugarcane.set_index(['CD_MUN', 'YEAR'])

In [6]:
# Adicionar a coluna do ano
data_sugar['YEAR'] = year_sugar

In [7]:
# Info
data_sugar.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10185 entries, (3500105, 2000) to (3557303, 2020)
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Município  10185 non-null  object  
 1   value      9525 non-null   float64 
 2   AET        10185 non-null  int64   
 3   NDVI       10185 non-null  int64   
 4   NPP        10185 non-null  int64   
 5   SOIL       10185 non-null  int64   
 6   TMIN       10185 non-null  int64   
 7   TMMX       10185 non-null  int64   
 8   PR         10185 non-null  int64   
 9   YEAR       10185 non-null  category
dtypes: category(1), float64(1), int64(7), object(1)
memory usage: 777.4+ KB


In [8]:
# Selecionar as variáveis que serão utilizadas na modelagem
exog_vars_sugar = ['AET', 'NDVI', 'NPP', 'SOIL', 'TMIN', 'TMMX', 'PR','YEAR']

In [9]:
# Adicionar a constante
exog_sugar = sm.add_constant(data_sugar[exog_vars_sugar])
exog_sugar.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10185 entries, (3500105, 2000) to (3557303, 2020)
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   const   10185 non-null  float64 
 1   AET     10185 non-null  int64   
 2   NDVI    10185 non-null  int64   
 3   NPP     10185 non-null  int64   
 4   SOIL    10185 non-null  int64   
 5   TMIN    10185 non-null  int64   
 6   TMMX    10185 non-null  int64   
 7   PR      10185 non-null  int64   
 8   YEAR    10185 non-null  category
dtypes: category(1), float64(1), int64(7)
memory usage: 697.8 KB


In [11]:
# Criar e rodar o modelo - Simple OLS
mod_sugar = PooledOLS(data_sugar.value, exog_sugar)
pooled_res_sugar = mod_sugar.fit()
print(pooled_res_sugar)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  value   R-squared:                        0.1264
Estimator:                  PooledOLS   R-squared (Between):              0.2293
No. Observations:                9525   R-squared (Within):               0.0785
Date:                Wed, Jul 20 2022   R-squared (Overall):              0.1264
Time:                        04:50:41   Log-likelihood                -1.031e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      50.884
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                 F(27,9497)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             50.884
                            

In [14]:
# Criar e rodar o modelo  - Random effects
mod_random_sugar = RandomEffects(data_sugar.value, exog_sugar)
re_res_sugar = mod_random_sugar.fit()
print(re_res_sugar)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  value   R-squared:                        0.1089
Estimator:              RandomEffects   R-squared (Between):              0.2085
No. Observations:                9525   R-squared (Within):               0.0829
Date:                Wed, Jul 20 2022   R-squared (Overall):              0.1226
Time:                        04:51:37   Log-likelihood                -1.018e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      43.002
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                 F(27,9497)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             35.122
                            

In [15]:
# Decomponer a variancia
re_res_sugar.variance_decomposition

Effects                   3.570774e+07
Residual                  1.115000e+08
Percent due to Effects    2.425671e-01
Name: Variance Decomposition, dtype: float64

Rodar o modelo sem considerar o ano (Between OLS)

In [17]:
# Excluir o ano das variavéis
exog_vars_sugar_sem_year = ['AET', 'NDVI', 'NPP', 'SOIL', 'TMIN', 'TMMX', 'PR']

In [18]:
# Adicionar a constante
exog_sem_year = sm.add_constant(data_sugar[exog_vars_sugar_sem_year])

In [20]:
# Criar e rodar o modelo sem o efeito do ano
mod_sem_year = BetweenOLS(data_sugar.value, exog_sem_year)
be_res_sugar = mod_sem_year.fit(reweight = True)
print(be_res_sugar)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                  value   R-squared:                        0.2332
Estimator:                 BetweenOLS   R-squared (Between):              0.2325
No. Observations:                 485   R-squared (Within):              -0.3028
Date:                Wed, Jul 20 2022   R-squared (Overall):             -0.1332
Time:                        04:53:24   Log-likelihood                   -4938.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      62.723
Entities:                         485   P-value                           0.0000
Avg Obs:                       19.639   Distribution:                   F(7,477)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             20.722
                            

Comparar os modelos

In [22]:
print(compare({"BE": be_res_sugar,'RE': re_res_sugar, "Pooled": pooled_res_sugar}))

                        Model Comparison                       
                                BE              RE       Pooled
---------------------------------------------------------------
Dep. Variable                value           value        value
Estimator               BetweenOLS   RandomEffects    PooledOLS
No. Observations               485            9525         9525
Cov. Est.               Unadjusted      Unadjusted   Unadjusted
R-squared                   0.2332          0.1089       0.1264
R-Squared (Within)         -0.3028          0.0829       0.0785
R-Squared (Between)         0.2325          0.2085       0.2293
R-Squared (Overall)        -0.1332          0.1226       0.1264
F-statistic                 62.723          43.002       50.884
P-value (F-stat)            0.0000          0.0000       0.0000
===================== ============ =============== ============
const                    1.862e+05       1.118e+05     1.46e+05
                          (9.5455)      